# Evaluate CTGAN on eICU cardiorespiratory past history data

In [2]:
import os
import numpy as np
import pandas as pd
import sys
import ctgan
from ctgan import CTGANSynthesizer
#from ctgan import load_demo

# hide warnings
import warnings
warnings.filterwarnings("ignore")

In [3]:
os.getcwd()

'c:\\Users\\dianam\\Documents\\jlab_datascience\\PlayGround\\DS6050\\CTGAN_Distribution'

In [4]:
# load the CTGAN demo data
data = pd.read_csv('examples\csv\patients_allpasthistory_onehot_cardiorespiratory.csv')
data.columns

Index(['age', 'hospitaldischargeoffset', 'CHF', 'COPD_moderate',
       'COPD_nolimitations', 'COPD_severe', 'NoHealthProblems', 'asthma',
       'homeoxygen', 'hypertensionrequiringtreatment',
       'restrictivepulmonarydisease', 'All', 'ethnicity_AfricanAmerican',
       'ethnicity_Asian', 'ethnicity_Caucasian', 'ethnicity_Hispanic',
       'ethnicity_NativeAmerican', 'ethnicity_Other/Unknown', 'gender_Female',
       'gender_Male', 'dischargestatus_Alive', 'dischargestatus_Expired'],
      dtype='object')

In [5]:
os.chdir('../eICU_gan/dataprep')
os.getcwd()


'c:\\Users\\dianam\\Documents\\jlab_datascience\\PlayGround\\DS6050\\eICU_gan\\dataprep'

In [6]:
from create_eICU_data import reverse_one_hot

In [7]:
os.chdir('../../CTGAN_Distribution')
os.getcwd()

'c:\\Users\\dianam\\Documents\\jlab_datascience\\PlayGround\\DS6050\\CTGAN_Distribution'

In [8]:
new_df = reverse_one_hot(data, columns=['ethnicity_AfricanAmerican',
       'ethnicity_Asian', 'ethnicity_Caucasian', 'ethnicity_Hispanic',
       'ethnicity_NativeAmerican', 'ethnicity_Other/Unknown'], prefix='ethnicity_', new_col='ethnicity')
#new_df.head()

new_df = reverse_one_hot(new_df, columns=['gender_Female','gender_Male'], prefix='gender_', new_col='gender')

new_df = reverse_one_hot(new_df, columns=['dischargestatus_Alive', 'dischargestatus_Expired'], prefix='dischargestatus_', new_col='dischargestatus')

new_df.head()


,age,hospitaldischargeoffset,CHF,COPD_moderate,COPD_nolimitations,COPD_severe,NoHealthProblems,asthma,homeoxygen,hypertensionrequiringtreatment,restrictivepulmonarydisease,All,ethnicity,gender,dischargestatus
0,87,2616,0,0,0,0,1,0,0,0,0,0,Caucasian,Female,Alive
1,76,1218,0,0,0,0,1,0,0,0,0,0,Caucasian,Male,Alive
2,34,1138,0,0,0,0,1,0,0,0,0,0,Caucasian,Female,Alive
3,61,5263,0,0,0,0,1,0,0,0,0,0,Caucasian,Male,Alive
4,55,1086,0,0,0,0,1,0,0,0,0,0,Caucasian,Female,Alive


In [9]:
print(new_df.columns)
# set the names of the demo data columns that are discrete

Index(['age', 'hospitaldischargeoffset', 'CHF', 'COPD_moderate',
       'COPD_nolimitations', 'COPD_severe', 'NoHealthProblems', 'asthma',
       'homeoxygen', 'hypertensionrequiringtreatment',
       'restrictivepulmonarydisease', 'All', 'ethnicity', 'gender',
       'dischargestatus'],
      dtype='object')


In [14]:
# save the eICU data without one hot encoded values so we can re-use it
new_df.to_csv('examples/csv/patients_cardiorespiratory_textcats.csv', index=False)

In [10]:
discrete_columns = [
    'CHF', 'COPD_moderate',
       'COPD_nolimitations', 'COPD_severe', 'NoHealthProblems', 'asthma',
       'homeoxygen', 'hypertensionrequiringtreatment',
       'restrictivepulmonarydisease', 'All', 'ethnicity', 'gender',
       'dischargestatus'
]

In [11]:
# initialze the CTGAN for 10 epochs, verbose, and both the generator and discriminator distributed
ctgan = CTGANSynthesizer(epochs=1000, verbose=True, distribute_g=True, distribute_d=True)

# fit the data
ctgan.fit(new_df, discrete_columns)
# create some samples
samples = ctgan.sample(1000)
# print the head of the generated dataframe
samples.head()

ctgan device:  cpu
ctgan device count:  0
ctgan generator distributed:  False
ctgan discriminator distributed:  False
Epoch 1, Loss G:  0.9114,Loss D: -0.0913
Epoch 2, Loss G:  0.8673,Loss D: -0.2083
Epoch 3, Loss G:  0.7017,Loss D: -0.3259
Epoch 4, Loss G:  0.4992,Loss D: -0.4073
Epoch 5, Loss G:  0.2347,Loss D: -0.3961
Epoch 6, Loss G: -0.2609,Loss D: -0.2668
Epoch 7, Loss G: -0.3596,Loss D: -0.2379
Epoch 8, Loss G: -0.5159,Loss D: -0.1781
Epoch 9, Loss G: -0.5248,Loss D: -0.1375
Epoch 10, Loss G: -0.5667,Loss D: -0.0519
Epoch 11, Loss G: -0.3998,Loss D:  0.0295
Epoch 12, Loss G: -0.4712,Loss D:  0.0361
Epoch 13, Loss G: -0.5186,Loss D:  0.0617
Epoch 14, Loss G: -0.3945,Loss D:  0.0962
Epoch 15, Loss G: -0.5083,Loss D:  0.1113
Epoch 16, Loss G: -0.5252,Loss D:  0.0692
Epoch 17, Loss G: -0.2443,Loss D:  0.1210
Epoch 18, Loss G: -0.2218,Loss D:  0.1473
Epoch 19, Loss G: -0.2474,Loss D:  0.0380
Epoch 20, Loss G: -0.0915,Loss D:  0.0065
Epoch 21, Loss G: -0.2219,Loss D:  0.0190
Epoch 22,

,age,hospitaldischargeoffset,CHF,COPD_moderate,COPD_nolimitations,COPD_severe,NoHealthProblems,asthma,homeoxygen,hypertensionrequiringtreatment,restrictivepulmonarydisease,All,ethnicity,gender,dischargestatus
0,70,14384,1,0,0,0,0,0,0,1,0,3,Caucasian,Male,Expired
1,18,16217,0,0,0,0,0,0,0,0,0,2,Caucasian,Male,Alive
2,30,32030,0,0,0,0,0,0,0,0,0,1,Caucasian,Male,Expired
3,58,3048,0,0,1,1,0,0,0,0,0,2,Caucasian,Male,Expired
4,66,7632,0,0,0,1,0,0,1,1,0,6,Caucasian,Male,Alive


# need to save results of various data quality tests for different numbers of epochs

In [12]:
# create the dataframe format and csv file to save evaluations of the CTGAN
# columns will be number epoch, generator loss, discriminator loss, number of generated samples, CSTest Value, KSTest Value, 
# MulticlassDecisionTreeClassifier for COPD_severe, MulticlassMLPClassifier for COPD_severe, 
# MulticlassDecisionTreeClassifier for CHF, MulticlassMLPClassifier for CHF
# MulticlassDecisionTreeClassifier for dischargestatus, MulticlassMLPClassifier for dischargestatus
# MulticlassDecisionTreeClassifier for COPD_severe, MulticlassMLPClassifier for COPD_severe - real train/test split, 
# MulticlassDecisionTreeClassifier for CHF, MulticlassMLPClassifier for CHF - real train/test split, 
# MulticlassDecisionTreeClassifier for dischargestatus, MulticlassMLPClassifier for dischargestatus - real train/test split, 

In [13]:
from sdv.evaluation import evaluate
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt